# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [15]:
#Your code here
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
import scipy.stats as stats
import numpy as np

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Display basic information
print(df.info())
print(df.head())

# Unique users count
print(f"Total unique users: {df['id'].nunique()}")

# Action counts
print(df['action'].value_counts())

# Check if any users clicked without viewing
viewers = df[df['action'] == 'view']['id'].unique()
clicks_without_views = df[(df['action'] == 'click') & (~df['id'].isin(viewers))]
print(f"Users who clicked without viewing: {clicks_without_views['id'].nunique()}")

# Control vs. Experiment group distribution
print(df['group'].value_counts())

# Check for users appearing in both groups
user_groups = df.groupby('id')['group'].nunique()
overlap_users = user_groups[user_groups > 1]
print(f"Users appearing in both groups: {overlap_users.count()}")

# Display some overlapping user examples (if any)
if not overlap_users.empty:
    print(df[df['id'].isin(overlap_users.index)].head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
Total unique users: 6328
action
view     6328
click    1860
Name: count, dtype: int64
Users who clicked without viewing: 0
group
control       4264
experiment    3924
Name: count, dtype: int64
Users appearing in both groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [16]:
#Your code here
# Pivot data to create a contingency table
contingency_table = pd.crosstab(df['group'], df['action'])

# Extract values: views and clicks for each group
clicks_control = contingency_table.loc['control', 'click']
views_control = contingency_table.loc['control', 'view']
clicks_experiment = contingency_table.loc['experiment', 'click']
views_experiment = contingency_table.loc['experiment', 'view']

# Create a contingency matrix
contingency_matrix = [[clicks_control, views_control - clicks_control], 
                      [clicks_experiment, views_experiment - clicks_experiment]]

# Perform Chi-Squared test
chi2, p, dof, expected = stats.chi2_contingency(contingency_matrix)

# Print results
print(f"Chi-Squared Statistic: {chi2:.4f}")
print(f"P-value: {p:.4f}")

# Interpret the results
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: The experimental homepage had a significant effect.")
else:
    print("Fail to reject the null hypothesis: No significant difference between the two pages.")


Chi-Squared Statistic: 6.7129
P-value: 0.0096
Reject the null hypothesis: The experimental homepage had a significant effect.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [17]:
#Your code here
# Get total users per group
control_users = df[df['group'] == 'control']['id'].nunique()
experiment_users = df[df['group'] == 'experiment']['id'].nunique()

# Get number of clicks per group
control_clicks = df[(df['group'] == 'control') & (df['action'] == 'click')]['id'].nunique()
experiment_clicks = df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].nunique()

# Calculate click-through rate for control group
ctr_control = control_clicks / control_users

# Calculate expected clicks in the experiment group
expected_experiment_clicks = ctr_control * experiment_users

print(f"Control Group CTR: {ctr_control:.4f}")
print(f"Expected Clicks in Experiment Group: {expected_experiment_clicks:.2f}")
print(f"Actual Clicks in Experiment Group: {experiment_clicks}")

Control Group CTR: 0.2797
Expected Clicks in Experiment Group: 838.02
Actual Clicks in Experiment Group: 928


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [18]:
#Your code here
# Calculate variance and standard deviation
variance = experiment_users * ctr_control * (1 - ctr_control)
std_dev = np.sqrt(variance)

# Compute Z-score
z_score = (experiment_clicks - expected_experiment_clicks) / std_dev

print(f"Standard Deviation: {std_dev:.4f}")
print(f"Z-score: {z_score:.4f}")

Standard Deviation: 24.5685
Z-score: 3.6625


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [19]:
#Your code here
from scipy.stats import norm

# Calculate p-value from the z-score
p_value = 2 * norm.sf(abs(z_score))

print(f"P-value: {p_value:.4f}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: The experimental homepage had a significant effect.")
else:
    print("Fail to reject the null hypothesis: No significant difference between the two pages.")

P-value: 0.0002
Reject the null hypothesis: The experimental homepage had a significant effect.


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [20]:
# Step 1: Calculate click-through rates
control_users = df[df['group'] == 'control']['id'].nunique()
experiment_users = df[df['group'] == 'experiment']['id'].nunique()
control_clicks = df[(df['group'] == 'control') & (df['action'] == 'click')]['id'].nunique()
experiment_clicks = df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].nunique()

ctr_control = control_clicks / control_users
expected_experiment_clicks = ctr_control * experiment_users

# Step 2: Chi-Squared Test
contingency_matrix = [[control_clicks, control_users - control_clicks], 
                      [experiment_clicks, experiment_users - experiment_clicks]]
chi2, p_chi2, dof, expected = stats.chi2_contingency(contingency_matrix)

# Step 3: Z-Test using binomial variance
variance = experiment_users * ctr_control * (1 - ctr_control)
std_dev = np.sqrt(variance)
z_score = (experiment_clicks - expected_experiment_clicks) / std_dev
p_z = 2 * stats.norm.sf(abs(z_score))

# Print results
print(f"Chi-Squared Test p-value: {p_chi2:.4f}")
print(f"Z-Test p-value: {p_z:.4f}")

# Interpretation
alpha = 0.05
if p_chi2 < alpha and p_z < alpha:
    print("Both tests agree: The experimental homepage had a significant effect.")
elif p_chi2 >= alpha and p_z >= alpha:
    print("Both tests agree: No significant difference between the two pages.")
else:
    print("Results are inconsistent. Further investigation is needed.")

Chi-Squared Test p-value: 0.0096
Z-Test p-value: 0.0002
Both tests agree: The experimental homepage had a significant effect.


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.